In [1]:
import random
n=12
V = [(random.uniform(5,75),random.uniform(5,55)) for i in range(n)]
import matplotlib.pyplot as plt
alpha=2

In [2]:
V

[(15.977031306544324, 43.70440930047421),
 (17.22807058322337, 28.50464207970468),
 (71.67709404991697, 43.17396798057854),
 (51.71967501677885, 25.147031232398472),
 (16.255997763820947, 44.86591843899375),
 (38.5711128766455, 11.303429241786034),
 (40.21274745799892, 49.146523380227535),
 (39.212823820392174, 5.798467708874728),
 (73.91660711929025, 37.17972954895307),
 (35.106622859639515, 42.512696938574365),
 (14.097420753739593, 22.306489019254386),
 (59.69416594441446, 11.186987974143054)]

In [3]:
alpha=2
def t_cost(s,t): return ((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5)
def d_cost(s,t): return (1/alpha)*(((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5))

In [4]:
def tpath_pathcost(start,end,tpath):
    pathcost=0
    a=tpath.index(start)
    b=tpath.index(end)
    if a<b:
        for i in range(a,b):
            pathcost=pathcost+t_cost(V[tpath[i]],V[tpath[(i+1)%len(tpath)]])
    if a>b:
        for i in range(a,len(tpath)):
            pathcost=pathcost+t_cost(V[tpath[i]],V[tpath[(i+1)%len(tpath)]])
        for i in range(b):
            pathcost=pathcost+t_cost(V[tpath[i]],V[tpath[(i+1)%len(tpath)]])
    if a==b:
        pathcost=0
            
    return pathcost

In [5]:
def center(V):
    x=[]
    y=[]
    for i in range(n):
        x.append(V[i][0])
        y.append(V[i][1])
    return [sum(x)/n,sum(y)/n]

In [6]:
import itertools
import matplotlib.pyplot as plt
def two_opt_truck(ps,truck_nodes):
    n = len(ps)
    res = list(range(n))
    update = True
    while update:
        update = False
        for (i,j) in itertools.combinations(range(n),2):
            if ((t_cost(ps[res[i]],ps[res[i+1]])+t_cost(ps[res[j]],ps[res[(j+1)%n]]))>
                (t_cost(ps[res[i]],ps[res[j]])+t_cost(ps[res[i+1]],ps[res[(j+1)%n]]))):
                res[i+1:j+1] = res[j:i:-1]
                update = True
                
    path=[truck_nodes[i] for i in res]

    return path

In [7]:
def dp_truck(ps,truck_nodes):
    length = {} # length[(u,S)]: u を始点とし S の点すべてを回る最小経路長、ディクショナリ
    route = {} # route[(u,S)]: 最小経路長を達成するためのルート、ディクショナリ
    v = ps[0]
    n = len(ps)
    for i in range(1,n+1):
        for a in itertools.combinations(range(n),i):
            S = frozenset(a)
            for j in S:
                u = ps[j]
                if i==1:
                    length[(j,S)] = t_cost(v,u)
                    route[(j,S)] = [j]
                else:
                    Sj = S-set([j]) # S から j を除いたもの
                    k=min(Sj,key=lambda k: length[(k,Sj)]+t_cost(ps[k],u))#Sjの中でlength[(k,Sj)]+dist(ps[k],u)が最小のものを返している
                    length[(j,S)] = length[(k,Sj)]+t_cost(ps[k],u)
                    route[(j,S)] = route[(k,Sj)]+[j]
    path=[truck_nodes[i] for i in route[(0,frozenset(range(n)))]]
    
    return path 

In [8]:
#トラックのルートをdpで求める
import itertools
def newalgo_dp_sub(V):
    n=len(V)
    ndmin=2
    ndmax=n//2
    m=1
    c=center(V)
    D={}
    result=[]
    for nd in range(ndmin,ndmax+1):
        length={i:t_cost(c,V[i]) for i in range(n)}
        drone_nodes=[]
        truck_nodes=[]
        drone_subnodes=[]
        initial_drone=[max(length, key=length.get)]
        del length[max(length, key=length.get)]
        for i in range(nd+m):
            drone_subnodes.append(max(length, key=length.get))
            del length[max(length, key=length.get)]
        for s in itertools.combinations(drone_subnodes,nd-1):
            drone_nodes=initial_drone+list(s)
            truck_nodes=list(set(range(n))-set(drone_nodes))


            T={}
            t=[V[i] for i in truck_nodes]
            tpath=two_opt_truck(t,truck_nodes)
            for drone in drone_nodes:
                for (start,end) in itertools.permutations(tpath,2):
                    drone_cost=d_cost(V[start],V[drone])+d_cost(V[drone],V[end])
                    truck_cost=tpath_pathcost(start,end,tpath)
                    T[start,drone,end]=max(drone_cost,truck_cost)


            #ドローンとトラックが一緒に移動する場合は考えない
            dc=[b for b in itertools.permutations(drone_nodes,nd)]
            tc=[a for a in itertools.combinations(tpath,nd)]
            for comb in itertools.product(tc,dc):
                d=0
                for j in range(nd):
                    d=d+T[comb[0][j],comb[1][j],comb[0][(j+1)%nd]]
                D[comb]=d

            result.append(min(D.values()))      

    res=min(result)
    return res

In [9]:
newalgo_dp_sub(V)

130.70812659342926